## 0- Importation des bibliotheque necessaire

#### on va utiliser flask comme un framework de backend et plotly pour la visualization et ...

In [1]:
from flask import Flask, request, jsonify, session
import plotly.graph_objects as go
from flask_cors import CORS
from itertools import combinations, groupby, islice
import networkx as nx
import random
from math import gamma
import numpy as np
import csv
import threading
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Manager
from time import time

In [2]:
app = Flask(__name__)
CORS(app)

In [3]:
def generate_edges(startindex,endindex, n, p,G,pos):
    for i in range(startindex,endindex):
         for j in range(i+1, n):
            distance = ((pos[i][0] - pos[j][0])**2 + (pos[i][1] - pos[j][1])**2)**0.5
            if distance <= p:
                G.add_edge(i, j, distance=random.randint(1, 10), charge=random.randint(1, 10))
                

def generate_random_graph_with_threads(n, p,nodes_per_thread=100):
    
    """
    Generates a random undirected graph, similarly to an Erdős-Rényi graph, but enforcing that the resulting graph 
    is connected
    
    """
    G = nx.Graph()
    G.add_nodes_from(range(n))
    pos = {node: (random.random(), random.random()) for node in range(n)}
    nx.set_node_attributes(G, pos, 'pos')
    if p <= 0:
        return G
    if p > 1:
        return nx.complete_graph(n, create_using=G)
    
    
    nbth = n // nodes_per_thread
    if nbth == 0:
        nbth = 1
    threads = []
    for i in range(nbth):
        startindex = i * nodes_per_thread
        endindex = startindex + nodes_per_thread
        if i == nbth - 1:
            endindex = n
        
        th = threading.Thread(target=generate_edges, args=(startindex,endindex, n, p,G,pos))
        th.start()
        threads.append(th)
        
    for th in threads:
        th.join()
        
    return G

In [4]:
def get_fig(G):
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = {
        'x': edge_x,
        'y': edge_y,
        'line': {'width': 0.5, 'color': '#888'},
        'hoverinfo': 'none',
        'mode': 'lines'
    }

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)

    node_trace = {
        'x': node_x,
        'y': node_y,
        'mode': 'markers',
        'hoverinfo': 'text',
        'marker': {
            'showscale': True,
            'colorscale': 'YlGnBu',
            'reversescale': True,
            'color': [],
            'size': 10,
            'colorbar': {
                'thickness': 15,
                'title': 'Node Connections',
                'xanchor': 'left',
                'titleside': 'right'
            },
            'line_width': 2
        }
    }

    node_adjacencies = []
    node_text = []
    i=0
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append('Node Id: '+str(i)+' is of connections: ' + str(len(adjacencies[1])))
        i+= 1

    node_trace['marker']['color'] = node_adjacencies
    node_trace['text'] = node_text

#         data = [edge_trace, node_trace]
    layout = {
        'title': '<br>The Graph Generated',
        'titlefont_size': 16,
        'showlegend': False,
        'hovermode': 'closest',
        'margin': {'b': 20, 'l': 5, 'r': 5, 't': 40},
        'annotations': [
            {'text': "Choose your Source and Destination then click on Graph Navigator to find the optimal path for them",
             'showarrow': False,
             'xref': "paper", 'yref': "paper",
             'x': 0.005, 'y': -0.002}
        ],
        'xaxis': {'showgrid': False, 'zeroline': False, 'showticklabels': False},
        'yaxis': {'showgrid': False, 'zeroline': False, 'showticklabels': False}
    }

    return {"edge_trace": edge_trace,"node_trace": node_trace, "layout": layout}

In [5]:
def get_fig_with_path(G, best_path_text, best_path_info, path=[]):
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = {
            'x': edge_x,
            'y': edge_y,
            'line': {'width': 0.5, 'color': '#888'},
            'hoverinfo': 'none',
            'mode': 'lines'
        }


    # Separate the edges based on whether they are in the path or not
    edge_sets = [path, list(set(G.edges()) - set(path))]
    colors = ['#ff0000', '#888']  # Color for edges in the path and default color

    edge_traces = []
    for edge_set, c in zip(edge_sets, colors):
        edge_x = []
        edge_y = []
        for edge in edge_set:
            x0, y0 = G.nodes[edge[0]]['pos']
            x1, y1 = G.nodes[edge[1]]['pos']
            edge_x.append(x0)
            edge_x.append(x1)
            edge_x.append(None)
            edge_y.append(y0)
            edge_y.append(y1)
            edge_y.append(None)

        edge_traces.append({
            'x': edge_x, 
            'y': edge_y,
            'line':  {'width':0.5, 'color':c},
            'hoverinfo': 'none',
            'mode': 'lines'})

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)

    node_trace = {
        'x': node_x,
        'y': node_y,
        'mode': 'markers',
        'hoverinfo': 'text',
        'marker': {
            'showscale': True,
            'colorscale': 'YlGnBu',
            'reversescale': True,
            'color': [],
            'size': 10,
            'colorbar': {
                'thickness': 15,
                'title': 'Node Connections',
                'xanchor': 'left',
                'titleside': 'right'
            },
            'line_width': 2
        }
    }

    node_adjacencies = []
    node_text = []
    i = 0
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append('Node Id: '+str(i)+' is of connections: ' + str(len(adjacencies[1])))
        i += 1

    node_trace['marker']['color'] = node_adjacencies
    node_trace['text'] = node_text

#         data = [edge_trace, node_trace]
    layout = {
        'title': best_path_info,
        'titlefont_size': 16,
        'showlegend': False,
        'hovermode': 'closest',
        'margin': {'b': 20, 'l': 5, 'r': 5, 't': 40},
        'annotations': [
            {'text': best_path_text,
             'showarrow': False,
             'xref': "paper", 'yref': "paper",
             'x': 0.005, 'y': -0.002}
        ],
        'xaxis': {'showgrid': False, 'zeroline': False, 'showticklabels': False},
        'yaxis': {'showgrid': False, 'zeroline': False, 'showticklabels': False}
    }
    
#     data = edge_traces + [node_trace]

    return {"data": edge_traces + [node_trace], "layout": layout}

### Normalize Graph

In [6]:
def find_max_distance(edges):
    if not edges:
        return 0
    max_distance = max(data['distance'] for _, _, data in edges)
    return max_distance

def find_max_charge(edges):
    if not edges:
        return 0
    max_charge = max(data['charge'] for _, _, data in edges)
    return max_charge

def normalized_graph_edges(list_of_edges , max_distance , max_charge , startindex , endindex):
    for i in range(startindex , endindex):
        list_of_edges[i][2]['distance'] = round(list_of_edges[i][2]['distance'] / max_distance, 4)
        list_of_edges[i][2]['charge'] = round(list_of_edges[i][2]['charge'] / max_charge, 4)
    
def normalize_graph_attributes(G,edges_per_thread=150 ):
    normalized_graph = G.copy()
    
    with ThreadPoolExecutor() as executor:
        future_distances = executor.submit(find_max_distance, G.edges(data=True))
        future_charges = executor.submit(find_max_charge, G.edges(data=True))
    
    max_distance = future_distances.result()
    max_charge = future_charges.result()
    
    list_of_edges = list(normalized_graph.edges(data=True))
    
    num_edges = len(list_of_edges)
    nbth = num_edges // edges_per_thread 
    if nbth == 0:
        nbth = 1
    threads = []
    for i in range(nbth):
        
        startindex = i * edges_per_thread 
        endindex = startindex + edges_per_thread 
        if i == nbth - 1:
            endindex = num_edges
        th = threading.Thread(target=normalized_graph_edges, args=(list_of_edges , max_distance , max_charge,
                                                                   startindex,endindex))
        th.start()
        threads.append(th)
        
    for th in threads:
        th.join()
        
    new_normalized_graph = nx.Graph()    
    new_normalized_graph.add_edges_from(list_of_edges)
    return new_normalized_graph  

### Save graph information

In [7]:
def write_edge_info(list_of_edges , writer,startindex,endindex):
    for i in range(startindex,endindex):
        writer.writerow({'source': list_of_edges[i][0], 'target': list_of_edges[i][1], 
                         'distance': list_of_edges[i][2]['distance'], 'charge': list_of_edges[i][2]['charge']})
    
def save_graph_info(G,edges_per_thread=150 ):  
    edges_file = "graph_info.csv"
    with open(edges_file, 'w', newline='') as csvfile:
        fieldnames = ['source', 'target', 'distance', 'charge']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        
        list_of_edges = list(G.edges(data=True))
        
        num_edges = len(list_of_edges)
        nbth = num_edges // edges_per_thread 
        if nbth == 0:
            nbth = 1
        threads = []
        for i in range(nbth):
            
            startindex = i * edges_per_thread 
            endindex = startindex + edges_per_thread 
            if i == nbth - 1:
                endindex = num_edges
                
            th = threading.Thread(target=write_edge_info, args=(list_of_edges , writer,startindex,endindex))
            th.start()
            threads.append(th)
            
        for th in threads:
            th.join()

### Cuckoo search algorithm

In [8]:
def calculate_total_distance(graph, path):
        total_distance = 0
        for i in range(len(path) - 1):
            source_node = path[i]
            target_node = path[i + 1]
            total_distance += graph[source_node][target_node]['distance']
        
        return total_distance 
                             
def calculate_total_charge(graph, path):
        total_charge = 0
        for i in range(len(path) - 1):
            source_node = path[i]
            target_node = path[i + 1]
            total_charge += graph[source_node][target_node]['charge']
        
        return total_charge 
                             

def evaluate_fitness(graph, path):
    
    with ThreadPoolExecutor() as executor:
        future_distances = executor.submit(calculate_total_distance, graph,path)
        future_charges = executor.submit(calculate_total_charge, graph,path)
    
    total_distance  = future_distances.result()
    total_charge = future_charges.result()
    
                                          
    return total_distance + total_charge


def k_shortest_paths(G, source, target, k, weight=None):
    try :
        return list(
        islice(nx.shortest_simple_paths(G, source, target, weight=weight), k)
        )
    except nx.NetworkXNoPath: # If no path exists
            return [()]  

def cuckoo_generation(graph,population,not_used_population,n,Pa,shared_best_paths,startindex,endindex):
    for i in range(startindex , endindex) :
        
        # Evaluate fitness for each cuckoo's path
        fitness_list = [evaluate_fitness(graph, path) for path in population]

        # Find the index of the best cuckoo
        best_cuckoo_index = np.argmin(fitness_list)
        best_path = population[best_cuckoo_index]
        best_fitness = fitness_list[best_cuckoo_index]

            
        if not_used_population :
            # here we will generate a new cuckoo solution (egg)
            new_cuckoo = random.choice(not_used_population)
            
            # calculate the fitness of the new slution
            new_cuckoo_fitness = evaluate_fitness(graph, new_cuckoo)
            
            # Choose a nest randomly ( a nest is a random possible solution from the population )
            random_nest_index = random.randint(0, n - 1)

            # compare the fitness value of the new cuckoo egg and the randomly selected eg of the nest. then replace the solution if the new one is better
            if new_cuckoo_fitness < fitness_list[random_nest_index]:
                population[random_nest_index] = new_cuckoo
                not_used_population.remove(new_cuckoo)

            # In case we have an increased number of eggs in the nest, abandon a fraction Pa of worst nests. this is to mimic the fact that there is a probability of Pa for the host ird to figure out the cuckoo egg.
            if len(population) > n:
                num_to_abandon = int(Pa * n)
                indices_to_abandon = np.argpartition(fitness_list, num_to_abandon)[:num_to_abandon]
                for index in indices_to_abandon:
                    population[index] = random.choice(not_used_population)
                    
        # Update shared list value with latest best path
        shared_best_paths[0] = best_path

def cuckoo_search(graph, source, target, population_size=50, generations=300, Pa=0.25,iterations_per_thread=100):
    
    start_time = time ()

    # nbr of solution we are going to work with
    n = population_size
    max_iterations = generations
    # the number of solution we will use
    nbr_population = generations * 3

    all_population = k_shortest_paths(G, source, target, nbr_population)
    if( all_population == [()]):
        end_time = time()
        return [()],[] , end_time-start_time


    # we will work with 50 possible solution
    population = all_population[:n]
    # the rest of the possible solutions we will work with them by generating new solutions from them.
    not_used_population = all_population[n:]

    best_path = None
    best_fitness = float('-inf')

    threads = []
    manager = Manager()
    shared_best_paths = manager.list([None])
    
    nbth = max_iterations // iterations_per_thread
    if nbth == 0:
        nbth = 1
    for i in range(nbth):
        
        startindex = i * iterations_per_thread
        endindex = startindex + iterations_per_thread
        if i == nbth - 1:
            endindex = max_iterations
        th = threading.Thread(target=cuckoo_generation, args=(graph,population,not_used_population,n,Pa,shared_best_paths,
                                                             startindex , endindex))
        th.start()
        threads.append(th)
        
    for th in threads:
        th.join()
        
    # get the last best_path found
    best_path = shared_best_paths[0]   

    # Convert the best path to a list of edge tuples
    best_path_edges = [(best_path[i], best_path[i + 1]) for i in range(len(best_path) - 1)]
    
    end_time = time()
    execution_time = end_time-start_time
    return best_path_edges , best_path , execution_time

### Threads configuration

In [9]:
#set the number of nodes per thread to 100
nodes_per_thread = 100 
#set the number of edges per thread to 200
edges_per_thread = 200 
#set the number of iterations per thread to 100
iterations_per_thread = 100
# the probability of nodes being connected
probability = 0.1

G = None 

In [10]:
@app.route('/get_graph', methods=['POST'])
def get_graph_api():
    
    data = request.get_json()
    
    nodes = int(data)

    global G
    G = generate_random_graph_with_threads(nodes, probability)
    
    fig_data = get_fig(G)
    
    return jsonify(fig_data)

@app.route('/get_graph_with_path', methods=['POST'])
def get_graph_path_api():
    
    data = request.get_json()
    
    source = data.get('source', None)
    destination = data.get('destination', None)

    if source is not None and destination is not None:
        source_node = int(source)
        target_node = int(destination)
        
        global G
        
        if G == None:
            return jsonify({"path_exist": 0, "path_data": "Please generate a graph first!!"})
        
        normlized_graph = normalize_graph_attributes(G)
        best_pathtuple ,best_path , execution_time = cuckoo_search(normlized_graph, source_node, target_node,iterations_per_thread)
        
        if(best_path != []):
            print("############################################## Best path ##############################################")
            print(best_pathtuple)
            total_distance = calculate_total_distance(G, best_path)
            total_charge = calculate_total_charge(G, best_path)
            print("Total distance : ", total_distance)
            print("Total charge : ", total_charge)
            best_path_text = f'<br>Best Path from Node ID: {source_node} to Node ID: {target_node} is {best_path}. Total distance is = {total_distance} and consumed charge is = {total_charge}.'
            best_path_info = f'Execution Time of the Algorithm is: {execution_time:.4f} seconds.'

            fig_data_with_path = get_fig_with_path(G, best_path_text, best_path_info, best_pathtuple)

            return jsonify({"path_exist": 1, "path_data": fig_data_with_path})
        else :
            no_path_message = f'There is no path between  {source_node} and {target_node}'
            print(no_path_message)
            return jsonify({"path_exist": 0, "path_data": no_path_message})
            

        print("The execution time of the research phase is {:.4f} seconds".format(execution_time))
        
#         best_path_text = f'<br>Best Path from Node ID: {source_node} to Node ID: {target_node} is {best_path}. Total distance is = {total_distance} and consumed charge is = {total_charge}.'
#         best_path_info = f'Execution Time of the Algorithm is: {execution_time:.4f} seconds.'
        
#         fig_data_with_path = get_fig_with_path(G, best_path_text, best_path_info, best_pathtuple)
            
#         return jsonify(fig_data_with_path)
    else:
        return jsonify({'error': 'Source and destination must be provided as integers.'}), 400

In [ ]:
if __name__ == '__main__':
    app.run(port=5200)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5200
Press CTRL+C to quit
127.0.0.1 - - [26/Jan/2024 18:42:23] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 18:42:23] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 18:49:39] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 18:49:40] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 18:51:00] "OPTIONS /get_graph_with_path HTTP/1.1" 200 -
[2024-01-26 18:51:00,732] ERROR in app: Exception on /get_graph_with_path [POST]
Traceback (most recent call last):
  File "C:\Users\fouad\Desktop\Github Projects\Chatbot for Real Estate Price Prediction\github_env\lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\fouad\Desktop\Github Projects\Chatbot for Real Estate Price Prediction\github_env\lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\fouad\Desktop\Github Projects\Chatbo

############################################## Best path ##############################################
[(65, 25), (25, 113), (113, 99), (99, 131), (131, 69), (69, 133), (133, 170), (170, 135), (135, 34), (34, 58), (58, 110), (110, 137), (137, 23), (23, 62), (62, 199)]
Total distance :  66
Total charge :  71


127.0.0.1 - - [26/Jan/2024 19:20:24] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:20:25] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:20:58] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:20:58] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:21:18] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:21:19] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:22:24] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:22:25] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:22:42] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:22:43] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:22:56] "OPTIONS /get_graph_with_path HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:22:57] "POST /get_graph_with_path HTTP/1.1" 200 -


There is no path between  97 and 59


...

127.0.0.1 - - [26/Jan/2024 19:29:54] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:29:55] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:30:32] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:30:32] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:30:37] "OPTIONS /get_graph_with_path HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:30:59] "POST /get_graph_with_path HTTP/1.1" 200 -


############################################## Best path ##############################################
[(1, 195), (195, 169), (169, 79), (79, 59), (59, 160), (160, 114), (114, 7), (7, 91), (91, 2)]
Total distance :  25
Total charge :  24


127.0.0.1 - - [26/Jan/2024 19:31:36] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:31:36] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:32:04] "OPTIONS /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:32:05] "POST /get_graph HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2024 19:32:17] "OPTIONS /get_graph_with_path HTTP/1.1" 200 -


############################################## Best path ##############################################
[(1, 353), (353, 52), (52, 648), (648, 799)]
Total distance :  14
Total charge :  5


127.0.0.1 - - [26/Jan/2024 19:32:48] "POST /get_graph_with_path HTTP/1.1" 200 -


In [ ]:
!python --version